In [13]:
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
mx.random.seed(2)
ctx = mx.cpu()
with open("nlg/mifid2.txt") as f:
    mifid = f.read()

In [14]:
character_list = list(set(mifid))
vocab_size = len(character_list)
character_dict = {}
for e, char in enumerate(character_list):
    character_dict[char] = e
time_numerical = [character_dict[char] for char in mifid]

def one_hots(numerical_list, vocab_size=vocab_size):
    result = nd.zeros((len(numerical_list), vocab_size), ctx=ctx)
    for i, idx in enumerate(numerical_list):
        result[i, idx] = 1.0
    return result

def textify(embedding):
    result = ""
    indices = nd.argmax(embedding, axis=1).asnumpy()
    for idx in indices:
        result += character_list[int(idx)]
    return result

In [15]:
batch_size = 32
seq_length = 64
# -1 here so we have enough characters for labels later
num_samples = (len(time_numerical) - 1) // seq_length
dataset = one_hots(time_numerical[:seq_length*num_samples]).reshape((num_samples, seq_length, vocab_size))
num_batches = len(dataset) // batch_size
train_data = dataset[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
train_data = nd.swapaxes(train_data, 1, 2)

In [16]:
labels = one_hots(time_numerical[1:seq_length*num_samples+1])
train_label = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
train_label = nd.swapaxes(train_label, 1, 2)

num_inputs = vocab_size
num_hidden = 256
num_outputs = vocab_size

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxz = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxr = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxh = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whz = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whr = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whh = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01

########################
#  Bias vector for hidden layer
########################
bz = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
br = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bh = nd.random_normal(shape=num_hidden, ctx=ctx) * .01

########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_outputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_outputs, ctx=ctx) * .01

params = [Wxz, Wxr, Wxh, Whz, Whr, Whh, bz, br, bh, Why, by]

for param in params:
    param.attach_grad()

In [17]:
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear)) / temperature
    exp = nd.exp(lin)
    partition = nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

In [18]:
def gru_rnn(inputs, h, temperature=1.0):
    outputs = []
    for X in inputs:
        z = nd.sigmoid(nd.dot(X, Wxz) + nd.dot(h, Whz) + bz)
        r = nd.sigmoid(nd.dot(X, Wxr) + nd.dot(h, Whr) + br)
        g = nd.tanh(nd.dot(X, Wxh) + nd.dot(r * h, Whh) + bh)
        h = z * h + (1 - z) * g

        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear, temperature=temperature)
        outputs.append(yhat)
    return (outputs, h)

In [19]:
def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = nd.array([0.], ctx=ctx)
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad
def sample(prefix, num_chars, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    string = prefix

    #####################################
    # Prepare the prefix as a sequence of one-hots for ingestion by RNN
    #####################################
    prefix_numerical = [character_dict[char] for char in prefix]
    input = one_hots(prefix_numerical)

    #####################################
    # Set the initial state of the hidden representation ($h_0$) to the zero vector
    #####################################
    h = nd.zeros(shape=(1, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(1, num_hidden), ctx=ctx)

    #####################################
    # For num_chars iterations,
    #     1) feed in the current input
    #     2) sample next character from from output distribution
    #     3) add sampled character to the decoded string
    #     4) prepare the sampled character as a one_hot (to be the next input)
    #####################################
    for i in range(num_chars):
        outputs, h = gru_rnn(input, h, temperature=temperature)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].asnumpy())
        string += character_list[choice]
        input = one_hots([choice])
    return string

In [20]:
epochs = 50
moving_loss = 0.

learning_rate = 2.0

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 100 epochs.
    ############################
    if ((e+1) % 10 == 0):
        learning_rate = learning_rate / 2.0
    h = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    for i in range(num_batches):
        data_one_hot = train_data[i]
        label_one_hot = train_label[i]
        with autograd.record():
            outputs, h = gru_rnn(data_one_hot, h)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()

    print("Epoch %s. Loss: %s" % (e, moving_loss))
    print("")
    print(sample("Traders will", 1024, temperature=.1))

Epoch 0. Loss: 2.9113809377981266

Traders wille the the the the the the the the the the therentiontite the the the therentiontintiontis ontiontionses the the the the the the the the the the the the the the the the the the the theres the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the therentiontins antiontins the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the therentiontiontiontiontis ons the the the the the the theres ontintiontiontis onte the the therentiontis on the the the the the the the the the the the the the the the therentiontiontiontiontins the the the the the the the the the the the the the the the the theres the the the the the the theres the the the the the the the therention the theres thes the the the the the the the the the 

Epoch 8. Loss: 1.1744952459455613

Traders will be client of the conserted in a consuled and and request to the consertice of the contract of the consertice to the consertice of the protection of the conserted or the contract investment firm or professional consult of the consertation of the conselted or the contract of the protection of the contract of the protection of the contract of the consertice of the conser or the protection of the professional consult of the consertice of the protection of the conserted and the conserted in the contract investment firm which the contract investment firm may be client of the professional consult of the professional consuled and and request to the contract investment firm with the conserted or the contract investment firm or professional consuled and of the contract in a consuled to the consertation of the contract of the consertice of the contract of the contract investment firm shall be client of the professional consuled and and request to th

Epoch 16. Loss: 0.9648495151283648

Traders will be appropriate to the conditions for the protection of the client or any regard to the contract in a consolidated to the contract in accordance with the protection of the client or an investment firm shall be clients and the protection of the client or an investment firm shall be clients and the contract in the first subparagraph in accordance with the financial instruments and the client or an investment firm shall be clients and and the client or an investment firm shall be clients and the contract in a consolidated to the conselted by the contract in a consolidated to the contract in accordance with the provision of the client or an investment firm shall be clients and activities of the contract in accordance with the financial instruments and and related acts in accordance with the financial instruments and the client or an investment firm shall be clients and the client or to the contract in the financial instruments and access to t

Epoch 24. Loss: 0.8781695486314105

Traders will be applicable to the contract and related acts and the protection of the protection of the protection of clients and the consolidated take and protection of the protection of the protection of the client or adopt the requirements and of the protection of the client or professional client orders and the protection of the protection of the protection of the protection of the protection of the protection of the client or request the contract and the protection of the protection of the protection of the protection of the client or an investment firm or market operator or a protection of the protection of the protection of the client or administrative and account the client orders or participants of the regulated market and the protection of the protection of the protection of the client or the protection of the protection of the client or the protection of the client or administrative and account the client or adopt the request the contract 

Epoch 32. Loss: 0.8313062568734291

Traders will be able to require the client or professional client orders in the contract in accordance with Article 14 of Regulation (EU) No 1095/2010 of the European Parliament and of the Council of 1 July 2015.

Power is considered to be provided in the first subparagraph in accordance with Article 14(1) to (1) and (3) of Article 20(1) of Regulation (EU) No 1095/2010 of the European Parliament and of the Council of 2 July 2014 on the protection of the client or provided that the contract in accordance with Article 14(1) to (1) of Regulation (EU) No 1095/2010 of the European Parliament and of the Council of 1 July 2014 on the Council of 14 Article 14(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 24(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 29(1) a

Epoch 40. Loss: 0.8044165074287345

Traders will be able to require the contract in accordance with Article 14(1) of the European Parliament and of the Council of 14 Article 14(1), Article 20(1), Article 25(1), Article 20(1), Article 20(1), Article 24(1), Article 20(1), Article 20(1), Article 20(1), Article 24(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 25(1), Article 20(1), Article 20(1), Article 20(1), Article 13(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(4), Article 24(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 20(1), Article 24(1), Article 24(1), Article 2

Epoch 48. Loss: 0.7916625210372783

Traders will be able to ensure that the contract in accordance with Article 14(1) of the European Parliament and of the Council of 2 July 2015.

Power is delegated to the Commission shall be empowered to adopt delegated acts in accordance with Article 14(1) of this Directive or of Regulation (EU) No 600/2014 and the provision of investment firms and the contract in accordance with Article 14 of Regulation (EU) No 1095/2010.

Article 13(1) and (3) of Article 2(1) of Regulation (EU) No 600/2014 and the provision of investment firms and the provisions of this Directive and of the Council of 14 July 2015.

Power is delegated to the Commission shall be empowered to adopt delegated acts in accordance with Article 14 of Regulation (EU) No 600/2014 and of the Council of 2 July 2015.

Power is delegated to the Commission shall be empowered to adopt delegated acts in accordance with Article 14(1) of the European Parliament and of the Council of 2 July 2015.

P